In [6]:
!pip install praw transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.8 MB/s eta 0:00:00


In [7]:
import praw
from praw.models import MoreComments
import pandas as pd
import time
from transformers import pipeline

## Scraping

In [3]:
# Setup PRAW with Reddit API credentials
reddit = praw.Reddit(
    client_id="EHSVoqAsOGZNHtnry4ACoQ",
    client_secret="D_iEQYt3Jhq_pszBiTK8ju6GeJht4Q",
    password="gyhxa8-qeqqom-Nawfuj",
    user_agent="Turmerik Task by u/browniekanya",
    username="browniekanya",
    check_for_async=False
)

In [4]:
# List of subreddits to scrape
subreddits = [
    "health", "AskDocs", "medical", "clinicalresearch", "medicine",
    "cancer", "diabetes", "MultipleSclerosis", "Epilepsy",
    "ChronicPain", "IBD", "CrohnsDisease", "UlcerativeColitis",
    "Psoriasis", "eczema", "Allergies", "Asthma", "lupus", "mentalhealth",
    "depression", "BipolarReddit", "schizophrenia", "Anxiety", "ADHD", "autism",
    "addiction", "stopsmoking", "OpiatesRecovery", "alcoholism", "stopdrinking",
    "WeightLossAdvice", "loseit", "fitness", "HealthyFood", "nutrition",
    "Supplements", "intermittentfasting", "keto", "vegan", "vegetarian",
    "GERD", "stroke", "hypertension", "Dermatology", "SkincareAddiction",
    "WomensHealth", "menshealth", "healthcare", "publichealth", "globalhealth",
    "healthIT", "TeleMedicine", "HealthInsurance", "epidemiology",
    "biology", "genetics", "Vitiligo", "Oncology"
]

In [6]:
# Function to scrape data
def scrape_subreddits(subreddit_list, limit_per_subreddit=50, search_term="clinical trial"):
    all_posts = []
    for subreddit in subreddit_list:
        sub = reddit.subreddit(subreddit)
        for post in sub.hot(limit=limit_per_subreddit):
            post_dict = {
                'subreddit': subreddit,
                'title': post.title,
                'text': post.selftext,
                'id': post.id
            }
            all_posts.append(post_dict)

            # Scrape comments
            post.comments.replace_more(limit=0)
            for comment in post.comments.list():
              if search_term.lower() in comment.body.lower():
                all_posts.append({
                    'subreddit': subreddit,
                    'title': f"Comment on: {post.title}",
                    'text': comment.body,
                    'id': comment.id
                })
            time.sleep(1.1) # Sleep for slightly over 1 second between requests
    return pd.DataFrame(all_posts)

In [7]:
# Scrape data
data = scrape_subreddits(subreddits)
print("Number of posts and comments mentioning 'clinical trial':", len(data))
print(data.head())

Number of posts and comments mentioning 'clinical trial': 2925
  subreddit                                              title text       id
0    health  The federal government is putting up nearly $2...       1cp3eef
1    health  Doc who claimed COVID shots cause magnetism ge...       1coqcs9
2    health  A new COVID-19 variant FLiRT has emerged. Here...       1cp3zd0
3    health  Just a few days on night shift has "long-term"...       1copscb
4    health  Hallucinogen from Sonoran Desert toad venom sh...       1cov6mn


In [8]:
# Save the data to a CSV file
data.to_csv('reddit_scraped_data.csv', index=False)

## Sentiment Analysis

In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [26]:
import torch
from transformers import pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
emotion_pipeline = pipeline('text-classification', model='j-hartmann/emotion-english-distilroberta-base')

In [34]:
# Function to predict emotion using the pre-trained pipeline
def predict_emotion(row):
    if row['title'].startswith("Comment on:"):
        text_to_analyze = row['text']  # Use the text for comments
    else:
        text_to_analyze = row['title']  # Use the title otherwise

    text_to_analyze = str(text_to_analyze).strip()

    if not text_to_analyze:  # Check if the text is empty
        return ("Empty", 0.0)

    words = text_to_analyze.split()
    truncated_text = " ".join(words[:min(len(words), 514)])

    try:
        results = emotion_pipeline(truncated_text)
        return results[0]['label'], results[0]['score']
    except Exception as e:
        print(f"Error processing text: {e}")
        return ("Error", 0.0)

In [35]:
# Load the data
data = pd.read_csv('reddit_scraped_data.csv')

# Apply the emotion analysis function
data['emotion'], data['confidence'] = zip(*data.apply(predict_emotion, axis=1))

# Print the updated DataFrame
print(data.head())

# Save the updated DataFrame to a new CSV file
data.to_csv('emotion_analyzed_data.csv', index=False)

Error processing text: The expanded size of the tensor (610) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 610].  Tensor sizes: [1, 514]
  subreddit                                              title text       id  \
0    health  The federal government is putting up nearly $2...  NaN  1cp3eef   
1    health  Doc who claimed COVID shots cause magnetism ge...  NaN  1coqcs9   
2    health  A new COVID-19 variant FLiRT has emerged. Here...  NaN  1cp3zd0   
3    health  Just a few days on night shift has "long-term"...  NaN  1copscb   
4    health  Hallucinogen from Sonoran Desert toad venom sh...  NaN  1cov6mn   

   emotion  confidence  
0  neutral    0.938209  
1  neutral    0.668158  
2  neutral    0.839129  
3  neutral    0.954349  
4  neutral    0.477056  


In [36]:
data['emotion'].value_counts()

emotion
neutral     1724
sadness      361
fear         269
surprise     182
disgust      173
anger        115
joy          100
Error          1
Name: count, dtype: int64